In [73]:
!pip install qiskit

In [74]:
# import basics
import numpy as np
from random import randint
from math import gcd
# import Qiskit tools
from qiskit import Aer, transpile, assemble
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
# import plot tools
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

In [75]:
# number to factor
N = 15
# random number a in [2,N-1] wtih gcd(a,N)=1
n=0
while n == 0:
    a = randint(2, N-1)
    if gcd(a, N) == 1:
      n = 1
    print("**:", a, N, gcd(a, N))

**: 8 15 1


In [76]:
# modular exponentiation gates: p = 2^0, 2^1, .... , 2^(m-1)
def c_Uamod15(a, p):
    U = QuantumCircuit(4)
    # concatenate U-factors to form U^p
    for iteration in range(p):
        if a in [2,13]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if a in [7,8]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a in [4, 11]:
            U.swap(1,3)
            U.swap(0,2)
        if a in [7,11,13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = "{0}^{1} mod {2}".format(a, p, N)
    c_U = U.control()
    return c_U
# inverse QFT
def qft_dagger(n):
    qc = QuantumCircuit(n)
    for q in range(n//2):
        qc.swap(q, n-q-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT†"
    return qc

In [77]:
# Initialize registers and the quantum circuit
n_work = 4 # L
n_control = 2 * n_work + 1 # 2*L+1
c = QuantumRegister(n_control, name='c')
w = QuantumRegister(n_work, name='w')
cl  = ClassicalRegister(n_control, name='cl')
qc = QuantumCircuit(c, w, cl)
# Initialize control qubits
for q in range(n_control):
  qc.h(q)
# Populate work register with state |1>
qc.x(n_control)
# Controlled-U^p operations formed by concatenation
for k in range(n_control):
    qc.append(c_Uamod15(a, 2**k),
             [k] + [i+n_control for i in range(n_work)])
# Inverse-QFT
qc.append(qft_dagger(n_control), range(n_control))
# Measure control register
qc.measure(c, cl)
qc.draw(fold=-1)
plt.savefig('circuit_{0}.jpg'.format(a))
plt.show()

<Figure size 640x480 with 0 Axes>

In [78]:
!pip install qiskit-aer

In [79]:
# simulate
aer_sim = Aer.get_backend('aer_simulator')
t_qc = transpile(qc, aer_sim)
obj = assemble(t_qc)
results = aer_sim.run(obj, shots=1024).result()
counts = results.get_counts()
print(counts)
plot_histogram(counts, title='N = {0} a = {1}'.format(N, a), figsize=(6,8))
plt.savefig('hist_{0}.jpg'.format(a))
plt.show()

{'000000000': 248, '110000000': 256, '010000000': 272, '100000000': 248}


<ipython-input-79-e5773dc10f86>:5: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  results = aer_sim.run(obj, shots=1024).result()


<Figure size 640x480 with 0 Axes>

In [80]:
!pip install contfrac

In [81]:
# import packages
import contfrac
#
phi = (385, 512) # phi3=[0.110000001]_2=0.751953125=385/512
l_phi = '110000001' # GBA inserted
coefficients = list(contfrac.continued_fraction(phi))
convergents = list(contfrac.convergents(phi))
#
print("cont frac of phi:",coefficients)
print("convergents of phi:", convergents)
print(a)

cont frac of phi: [0, 1, 3, 31, 1, 3]
convergents of phi: [(0, 1), (1, 1), (3, 4), (94, 125), (97, 129), (385, 512)]
8


In [82]:
# import basics
import contfrac
from numpy import gcd
# construct decimal value of l_phi
n=0
l_tilde = 0
for l in l_phi[::-1]:
    n += 1
    l_tilde = l_tilde + 2**(n-1) * int(l)
print("l_measured   :", l_phi, l_tilde)
# construct decimal value of phi
n=0
phi_tilde = 0
for l in l_phi:
    n -= 1
    phi_tilde = phi_tilde + 2**n * int(l)
print("phi_phase_bin :", "0."+l_phi)
print("phi_phase_dec:", phi_tilde)
# express phi_tilde as a fraction
res = len(str(phi_tilde)) - 2 # subtract 2 for "0."
print("res:", res)
scale = 10**res # automated scale set by res
num = int(phi_tilde*scale)
den = int(scale)
# in lowest terms
c = gcd(num, den)
num = int(num / c)
den = int(den / c)
phi = (num, den)
print("phi:", phi)
# construct convergents for phi
coefficients = list(contfrac.continued_fraction(phi))
convergents = list(contfrac.convergents(phi))
print("cont frac of phi:",coefficients)
print("convergents of phi:", convergents)
# check convergents for solution
for conv in convergents:
    r = conv[1]
    test1 = r % 2 # 0 if r is even
    test2 = (a**int(r/2)-1) % N # 0 if a^r/2 is a trivial root
    test3 = (a**int(r/2)+1) % N # 0 if a^r/2 is a trivial root
    test4 = a**r % N # 1 if r is a solution
    print(test1, test2, test3, test4," conv:", conv, r, a)
    if (test1==0 and test2!=0 and test3!=0 and test4==1):
    #if (test1==0 and test2==0 and test3!=0 and test4==1): # GBA modified
        print("conv:", conv, "r =", r, ": factors")
        print("factor1:", gcd(a**int(r/2)-1, N))
        print("factor2:", gcd(a**int(r/2)+1, N))
    else:
        print("conv:", conv, "r =", r, ": no factors found")

l_measured   : 110000001 385
phi_phase_bin : 0.110000001
phi_phase_dec: 0.751953125
res: 9
phi: (385, 512)
cont frac of phi: [0, 1, 3, 31, 1, 3]
convergents of phi: [(0, 1), (1, 1), (3, 4), (94, 125), (97, 129), (385, 512)]
1 0 2 8  conv: (0, 1) 1 8
conv: (0, 1) r = 1 : no factors found
1 0 2 8  conv: (1, 1) 1 8
conv: (1, 1) r = 1 : no factors found
0 3 5 1  conv: (3, 4) 4 8
conv: (3, 4) r = 4 : factors
factor1: 3
factor2: 5
1 3 5 8  conv: (94, 125) 125 8
conv: (94, 125) r = 125 : no factors found
1 0 2 8  conv: (97, 129) 129 8
conv: (97, 129) r = 129 : no factors found
0 0 2 1  conv: (385, 512) 512 8
conv: (385, 512) r = 512 : no factors found
